<a href="https://colab.research.google.com/github/arpitvaghela/DSA_FL/blob/main/Copy_of_FL_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM time series prediction using Federated Learning

## Imports

In [7]:
!pip install --quiet --upgrade tensorflow_federated_nightly
!pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [8]:
import collections
import functools
import os
import time
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf

np.random.seed(0)

In [9]:
import tensorflow_federated as tff

# Test the TFF is working:
tff.federated_computation(lambda: 'Hello, World!')()

ImportError: ignored

## Data preprocessing


In [ ]:
#FM = np.fromfile('96_7_20db',dtype=np.float32)
N = 10**6
FM = np.random.randn(N) # normal distributed channel
FM.reshape(-1,1) 

### Bandpower Equation


In [ ]:
def bandpower(signal:np.array)->np.float:
  return np.mean(signal ** 2)

In [ ]:
bandpower(FM)

### awgn Function

Function to add noise to the signal resulting in given S/N ratio

In [ ]:
def awgn(signal:np.ndarray, desired_snr:int):
  """Add AWGN noise to generate signal with given SNR. 
  """
  # Converting the SNR from dB scale to linear scale
  snr_linear = math.pow(10, desired_snr / 10)
  
  # Standard normally distributed noise
  noise = np.random.randn(signal.shape[0], 1)
  
  # Using the boxed formula
  var_signal = bandpower(noise) * snr_linear
  
  # Normalizing the signal to have the given variance
  normalized_signal = math.sqrt(var_signal) * (signal / math.sqrt(bandpower(signal)))
  
  #print("SNR = " + str(10 * math.log10(bandpower(normalized_signal) / bandpower(noise))))
  
  return normalized_signal + noise

## Filtering Data

filtering data points to be in range $10^{-7}< signal< 1$

In [ ]:
FM = FM[np.logical_and(FM > math.pow(10, -7), FM < 1)]
FM = FM.reshape(FM.shape[0], 1)
print("Size of FM: " + str(FM.shape))

## Creating Dataset

- take datapoints of size, _samples*sample_size_

- add noise with desire snr

- sample = $[s_1,s_2,\cdots,s_N]$

- Energy detection = $\sum_{i=1}^{N}s_i^2$

- $X[j]$ = $\sum_{i=j*N+1}^{(j+1)*N}s_i^2$

In [ ]:
def create_dataset(signal, desired_snr, samples, sample_size):
  
  # Creating the signal with desired SNR
  snr_signal = awgn(signal[0:samples * sample_size], desired_snr)
  
  # Allocating zeros to the dataset
  X = np.zeros((samples, 1))
  
  for i in range(0, samples):
    
    # Extracting the sample based on sample size
    sampled_signal = snr_signal[i * sample_size : (i + 1) * sample_size]
    
    # Sorting the sampled signal
    sampled_signal = np.sort(sampled_signal, axis=0)
    
    # Energy detection
    E = np.sum(sampled_signal ** 2)
    
    # Assigning values to the dataset
    X[i][0] = E
  
  return X

In [ ]:
%%time
# sample dataset for SNR=4

print(create_dataset(FM[50000:], 4, 15000, 100).shape)

In [ ]:
def final_dataset(signal, snr_range, samples_per_snr, sample_size):
  X = {}
  
  for snr in snr_range:
    # Creating dataset for the given SNR
    X_snr = create_dataset(signal, snr, samples_per_snr, sample_size)
    
    # Indexing within the final dataset matrix X
    X[snr] = X_snr
  
  return X

## Generating White Noise Sequence 

In [ ]:
def create_noise_sequence(samples, sample_size):
  
  # Creating white noise sequence of variance 1
  noise = np.random.randn(samples * sample_size, 1)
  
  # Allocating zeros to the dataset
  X = np.zeros((samples, 1))
  
  for i in range(0, samples):
    
    # Extracting the sample based on sample size
    sampled_signal = noise[i * sample_size : (i + 1) * sample_size]
    
    # Sorting the sampled signal
    sampled_signal = np.sort(sampled_signal, axis=0)
    
    # Energy detection
    E = np.sum(sampled_signal ** 2)
    
    # Assigning values to the dataset
    X[i][0] = E
  
  return X

## DataSet LookBack for RNN

In [ ]:
# Function for Chaning the dataset for look back  #linear dataset initially
def create_look_back(X, look_back=1):
  
  # Look back dataset is initialized to be empty
  look_back_X = []
  
  for i in range(len(X) - look_back + 1):
    # Extracting an example from the dataset
    a = X[i:(i + look_back), :]
    
    a = a.flatten() # (For flattening) #1D list
    
    # Appending to the dataset
    look_back_X.append(a)
  
#  look_back_Y = []
    
  # Returning in numpy's array format
  return np.array(look_back_X)

In [ ]:
def dataset_look_back(X_tech, snr_range, look_back):
  X_tech_lb = {}
  
  # Look backs for all SNRs
  for snr in snr_range:
    X_tech_lb[snr] = create_look_back(X_tech[snr], look_back)
  
  return X_tech_lb

In [ ]:
def generate_Dataset(FM,snr_ratio=(0.8, 0.2),sample_size=100):
  lsnr_range=(-20,-4)
  hsnr_range=(-4,6)
  look_back = 2
  N = len(FM)
  
  no_of_sample_hsnr = int((N*snr_ratio[0])//((hsnr_range[1] - hsnr_range[0])/2))
  no_of_sample_lsnr = int((N*snr_ratio[1])//((lsnr_range[1] - lsnr_range[0])/2))
  
  X_FM = {**final_dataset(FM, range(lsnr_range[0],lsnr_range[1], 2),no_of_sample_lsnr, sample_size),
          **final_dataset(FM, range(hsnr_range[0],hsnr_range[1], 2), no_of_sample_hsnr, sample_size)}

  X_noise = create_noise_sequence(N, sample_size)

  # lookback

  X_FM_lb = dataset_look_back(X_FM, range(-20, 6, 2), look_back)
  X_noise_lb = create_look_back(X_noise, look_back)

  # final X_train and y

  X = X_FM_lb[-20]
  y = []

  for snr in range(-18, 6, 2):
    X = np.concatenate((X, X_FM_lb[snr]), axis=0)

  y = np.ones((X.shape[0], 1))
 
  # print(X.shape)
  # print(X_noise_lb.shape)
  X = np.concatenate((X, X_noise_lb), axis=0)
  y_train = np.concatenate((y, np.zeros((X_noise_lb.shape[0], 1))))

  
  # reshape
  X_train = np.reshape(X, (-1, 2, 1))
  Y_train = np.reshape(y_train,(-1,1,1))

  # index = [i for i in range(len(X_train))]
  # np.random.shuffle(index)

  # X_train = np.array([X_train[i] for i in index])
  # Y_train = np.array([Y_train[i] for i in index])
  
  return X_train,Y_train

 

In [ ]:
# created dataset
X,Y = generate_Dataset(FM)

In [ ]:
print("Shape of X,Y :",X.shape,Y.shape) # similar to the shape we have

# train,test split
train_size = int(len(X)*0.80)
test_size = len(X) - train_size

X_train, X_test = X[:train_size,:,:],X[train_size:,:,:]
Y_train, Y_test = Y[:train_size,:,:],Y[train_size:,:,:]


print(X_train.shape,X_test.shape)
print(Y_train.shape,Y_test.shape)


In [ ]:
X_train[0],Y_train[0]

## LSTM model and training

In [ ]:
model = tf.keras.models.Sequential([
      tf.keras.layers.LSTM(4,input_dim=1),
      tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,Y_train,validation_split=0.25,epochs=3,verbose=1)

model.evaluate(X_test,Y_test,verbose=1)

## LSTM model test

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

Y_pred = model.predict(X_test).ravel()
Y_pred.reshape(-1,1)
fpr, tpr, threshold = roc_curve(Y_test.reshape(-1,1), Y_pred) 
auc_tf = auc(fpr,tpr)
plt.plot(fpr, tpr, label='LSTM (area = {:.3f})'.format(auc_tf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend()
plt.show()

In [ ]:
model.predict(X_test[0:4])

## Federated Training and preprocessing centrailized data

### Generating a decentralized data

In [ ]:
T_CLIENTS = 500
CLIENT_SAMPLE_SIZE = len(X_train)/T_CLIENTS
step = len(X_train)/T_CLIENTS

data_fed = [ tf.data.Dataset.from_tensor_slices(
         {
          "value":X_train[int(i*step):int((i+1)*step)],
          "label":Y_train[int(i*step):int((i+1)*step)]
        }
     ) for i in range(T_CLIENTS)]

# client dataset can be accesed as data_fed[ CLIENT_ID ]
example_dataset = data_fed[0]

#   def preprocess(data_fed):
#   def batch_format_fn(ele):

example_element = next(iter(example_dataset))

print(example_element["value"].numpy()) # example_element[0] refers to X val
print(example_element["label"].numpy()) # example_element[1] refers to X val

In [ ]:
# preprocess

NUM_CLIENTS = T_CLIENTS
NUM_EPOCHS = 5
BATCH_SIZE = int(CLIENT_SAMPLE_SIZE//2)
SHUFFLE_BUFFER = 100
PREFETCH_BUFFER = 10

def preprocess(dataset):
  
  def batch_format_fn(element):
    return collections.OrderedDict(
        x=element["value"],
        y=element["label"]
    )
  return dataset.repeat(NUM_EPOCHS).shuffle(SHUFFLE_BUFFER).batch(BATCH_SIZE).map(batch_format_fn).prefetch(PREFETCH_BUFFER)


In [ ]:

preprocessed_example_dataset = preprocess(example_dataset)

sample_batch = tf.nest.map_structure(lambda x: x.numpy(),
                                     next(iter(preprocessed_example_dataset)))

sample_batch["x"].shape

In [ ]:
def make_federated_data(client_data,client_ids):
  return [
      preprocess(client_data[x])
      for x in client_ids
  ]


In [ ]:
sample_clients = [x for x in range(NUM_CLIENTS)]

federated_train_data = make_federated_data(data_fed, sample_clients)

print('Number of client datasets: {l}'.format(l=len(federated_train_data)))
print('First dataset: {d}'.format(d=federated_train_data[0]))


### Creating the Iterative process

In [ ]:
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

def model_fn():
  keras_model = tf.keras.models.Sequential([
      tf.keras.layers.LSTM(4,input_dim=1),
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return tff.learning.from_keras_model(
    keras_model,
    input_spec=preprocessed_example_dataset.element_spec,
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.BinaryAccuracy()]
  )

iterative_process = tff.learning.build_federated_averaging_process(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

str(iterative_process.initialize.type_signature)


In [ ]:
state = iterative_process.initialize()
tff_model = tf.keras.models.clone_model(model)
tff_model.compile(
    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.SGD(),
    metrics=[tf.keras.metrics.BinaryAccuracy()])

def keras_evaluate(state, round_num):
  state.model.assign_weights_to(tff_model)
  loss, accuracy = tff_model.evaluate(X_test,Y_test, steps=2, verbose=0)
  print('\tEval: loss={l:.3f}, accuracy={a:.3f}'.format(l=loss, a=accuracy))

### Iterations

In [ ]:
NUM_ROUNDS = 21
CLIENT_PER_ROUND = 25
for round_num in range(1, NUM_ROUNDS):
  # 500 clients are selected for training in various rounds
  # there are 10 rounds and each round takes data from 50 clients

  clients = [round_num*x for x in range(1,CLIENT_PER_ROUND)]
  federated_train_data = make_federated_data(data_fed, clients)
  
  print("ROUND ",round_num)
  keras_evaluate(state, NUM_ROUNDS + 1) # eval before hand
  state, metrics = iterative_process.next(state, federated_train_data)
  print('\tTrain: loss={}, accuracy={} '.format(round_num, metrics["train"]["loss"],metrics["train"]["binary_accuracy"]))

keras_evaluate(state, NUM_ROUNDS + 1)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# tensorflow LSTM 
Y_pred = model.predict(X_test).ravel()
Y_pred.reshape(-1,1)
fpr, tpr, threshold = roc_curve(Y_test.reshape(-1,1), Y_pred) 
auc_tf = auc(fpr,tpr)
plt.plot(fpr, tpr, label='TF (area = {:.3f})'.format(auc_tf))


# Federated Learning
state.model.assign_weights_to(tff_model)

Y_pred_tff = tff_model.predict(X_test).ravel()
Y_pred_tff.reshape(-1,1)
fpr_tff, tpr_tff, threshold_tff = roc_curve(Y_test.reshape(-1,1), Y_pred_tff) 
auc_tff = auc(fpr_tff,tpr_tff)
plt.plot(fpr_tff, tpr_tff,'k', label='FL (area = {:.3f})'.format(auc_tff))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend()
plt.show()

### Plots


#### ROC for various SNR  (TF vs TFF)

In [ ]:
# generating test-dataset for various snr
def generate_Data_dict(FM,snr_ratio=(0.6, 0.4),sample_size=100):
  lsnr_range=(-20,-4)
  hsnr_range=(-4,6)
  look_back = 2
  N = len(FM)
  
  no_of_sample_hsnr = int((N*snr_ratio[0])//((hsnr_range[1] - hsnr_range[0])/2))
  no_of_sample_lsnr = int((N*snr_ratio[1])//((lsnr_range[1] - lsnr_range[0])/2))
  
  X_FM = {**final_dataset(FM, range(lsnr_range[0],lsnr_range[1], 2),no_of_sample_lsnr, sample_size),
          **final_dataset(FM, range(hsnr_range[0],hsnr_range[1], 2), no_of_sample_hsnr, sample_size)}


  # lookback

  X_FM_lb = dataset_look_back(X_FM, range(-20, 6, 2), look_back)

  # final X_train and y

  data = {}

  for snr in range(-20, 6, 2):
    X = X_FM_lb[snr]
    y_ones = np.ones((X.shape[0], 1))

    X_noise = create_noise_sequence(len(X)*sample_size, sample_size)
    X_noise_lb = create_look_back(X_noise, look_back)
    X = np.concatenate((X, X_noise_lb), axis=0)
    Y = np.concatenate((y_ones, np.zeros((X_noise_lb.shape[0], 1))))

    # reshape
    X = np.reshape(X, (-1, 2, 1))
    Y= np.reshape(Y,(-1,1,1))

    index = [i for i in range(len(X))]
    np.random.shuffle(index)
    X_train = np.array([X[i] for i in index])
    Y_train = np.array([Y[i] for i in index])

    data[snr] = {
        "x":X,
        "y":Y
    }
  
  return data


In [ ]:
data_snr = generate_Data_dict(FM[int(-N*(0.01)):])
data_snr[-20]["x"].shape,data_snr[-20]["y"].shape

In [ ]:
# roc for various snr 

for snr_db in range(-20,6,2):
  # tensorflow_lstm
  Y_pred_tf = model.predict(data_snr[snr_db]["x"])
  Y_pred_tf.reshape(-1,1)
  fpr, tpr, threshold = roc_curve(data_snr[snr_db]["y"].reshape(-1,1), Y_pred_tf) 
  auc_tf = auc(fpr,tpr)
  plt.plot(fpr, tpr, label='TF (snr = {:.3f})'.format(snr_db))
  # tff_lstm
  # Y_pred_tff = tff_model.predict(data_dict[snr_db]["x"]).ravel()
  # Y_pred_tff.reshape(-1,1)
  # fpr_tff, tpr_tff, threshold_tff = roc_curve(data_dict[snr_db]["y"].reshape(-1,1), Y_pred_tff) 
  # auc_tff = auc(fpr_tff,tpr_tff)
  # plt.plot(fpr_tff, tpr_tff,'k', label='FL (snr = {:.3f})'.format(snr_db))


plt.legend()
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.show()

In [ ]:
# roc for various snr 

for snr_db in range(-20,6,2):
  # tensorflow_lstm
  # Y_pred_tf = model.predict(data_snr[snr_db]["x"])
  # Y_pred_tf.reshape(-1,1)
  # fpr, tpr, threshold = roc_curve(data_snr[snr_db]["y"].reshape(-1,1), Y_pred_tf) 
  # auc_tf = auc(fpr,tpr)
  # plt.plot(fpr, tpr, label='TF (snr = {:.3f})'.format(snr_db))
  # tff_lstm
  Y_pred_tff = tff_model.predict(data_snr[snr_db]["x"]).ravel()
  Y_pred_tff.reshape(-1,1)
  fpr_tff, tpr_tff, threshold_tff = roc_curve(data_snr[snr_db]["y"].reshape(-1,1), Y_pred_tff) 
  auc_tff = auc(fpr_tff,tpr_tff)
  plt.plot(fpr_tff, tpr_tff,'k', label='FL (snr = {:.3f})'.format(snr_db))


plt.legend()
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.show()

# Cooperative Spectrum Sensing using LSTM model

In [ ]:
# imports
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from typing import List

In [ ]:
# qfuncinv similar to one in matlab
def qfuncinv(x):
    return -1*norm.ppf(x)

In [ ]:
# initializing variable

L = 100      # sample size
N = 100     # number of simulations
nodes = 5     # number of Sec Users

SNR_db =  np.arange(-20,-18)    # snr range in dB
# SNR_lin = 10**(SNR_db/10)     # snr - linear

# Remove this part to include the ML model 

Pf = 0.2     # const probability of false alarm

Threshold = (qfuncinv(Pf)/math.sqrt(L)) + 1

In [ ]:
#  Simulation for Pd

# prediction based on lstm model
def detection_tf(energy_stat,energy_stat_pre) -> bool:
  return model.predict([[[energy_stat],[energy_stat_pre]]]).reshape((-1))[0] >= 0.5

def detection_tff(energy_stat,energy_stat_pre) -> bool:
  return tff_model.predict([[[energy_stat],[energy_stat_pre]]]).reshape((-1))[0] >= 0.5
# OR_based fusion 
def detect_func_OR(arr):
  return np.sum(arr) > 0

# AND_based fusion 
def detect_func_AND(arr:List[bool])-> np.ndarray:
  return np.multiply.reduce(arr) == 1

# Majority_based fusion
def detect_func_M(arr:List[bool])-> np.ndarray:
  return np.count_nonzero(arr)/len(arr) >= 0.5


Pd_OR_tf = np.array([])
Pd_AND_tf = np.array([])
Pd_M_tf = np.array([])

Pd_OR_tff = np.array([])
Pd_AND_tff = np.array([])
Pd_M_tff = np.array([])

SNR_db = [ i for i in range(-20,6,2)]
for snr_db in range(-20,6,2):

  detect_OR_tf = 0
  detect_AND_tf = 0
  detect_M_tf = 0

  detect_OR_tff = 0
  detect_AND_tff = 0
  detect_M_tff = 0


  print("At snr :",snr_db)

  Energy_stat_pre = np.array([])
  for _ in range(N):
    noise = np.random.randn(L,nodes) #nodesxL
    signal = math.sqrt(10**(snr_db/10))*np.random.randn(L,nodes)

    Rec_signal = signal + noise
    #print(noise,signal,Rec_signal)
        
    Energy_stat = np.sum(np.abs(Rec_signal**2) ,axis=0) 
           
    if not Energy_stat_pre.any():
      Energy_stat_pre = Energy_stat

    #print(Energy_stat[1],Energy_stat_pre[1])
    det_tf = [detection_tf(Energy_stat[x],Energy_stat_pre[x]) for x in range(nodes)]
    det_tff = [detection_tff(Energy_stat[x],Energy_stat_pre[x]) for x in range(nodes)]
    
    Energy_stat_pre = Energy_stat
    #print(detect)

    if detect_func_AND(det_tf):
      detect_AND_tf += 1

    if detect_func_AND(det_tff):
      detect_AND_tff += 1

    if detect_func_OR(det_tf):
      detect_OR_tf += 1

    if detect_func_OR(det_tff):
      detect_OR_tff += 1

    if detect_func_M(det_tf):
      detect_M_tf += 1

    if detect_func_M(det_tff):
      detect_M_tff += 1

  Pd_OR_tf = np.append(Pd_OR_tf,detect_OR_tf/N)
  Pd_AND_tf = np.append(Pd_AND_tf,detect_AND_tf/N)
  Pd_M_tf = np.append(Pd_M_tf,detect_M_tf/N)

  Pd_OR_tff = np.append(Pd_OR_tff,detect_OR_tff/N)
  Pd_AND_tff = np.append(Pd_AND_tff,detect_AND_tff/N)
  Pd_M_tff = np.append(Pd_M_tff,detect_M_tff/N)

#plot
plt.plot(SNR_db,Pd_OR_tf,'bo-',SNR_db,Pd_AND_tf,'ro-',SNR_db,Pd_M_tf,'ko-',SNR_db,Pd_OR_tff,'bx--',SNR_db,Pd_AND_tff,'rx--',SNR_db,Pd_M_tff,'kx--')
# plot properties
plt.xlabel("Signal To Noise Ratio (dB)")
plt.ylabel("Probability of Detection (P_d)")
plt.title("P_d v/s SNR (TF vs TFF comparision)")

plt.legend(["Pd_OR_tf","Pd_AND_tf","Pd_M_tf","Pd_OR_tff","Pd_AND_tff","Pd_M_tff"])
plt.grid()
plt.show()

In [ ]:
model.predict([[[100.2335050401152],[155.1100918561721]]])

In [ ]:
N = 100
SNR_db = np.arange(-20,5,2)
markers = [".","o","8","^"]
i = 0
for k in [1,2,5,10]: # number of nodes / SU
  print("K =",str(k))

  Pd_OR = np.array([])

  for snr_db in SNR_db:

    detect_OR = 0

    print("\tAt snr :",snr_db)

    Energy_stat_pre = np.array([])
    for _ in range(N):
      noise = np.random.randn(L,k)

      signal = math.sqrt(10**(snr_db/10))*np.random.randn(L,k)

      Rec_signal = signal + noise 

      Energy_stat = np.sum(np.abs(Rec_signal**2) ,axis=0) 

      if not Energy_stat_pre.any():
        Energy_stat_pre = Energy_stat

      detect = [detection(Energy_stat[x],Energy_stat_pre[x]) for x in range(k)]

      Energy_stat_pre = Energy_stat
      #print(detect)

      if detect_func_OR(detect):
        detect_OR += 1


    Pd_OR = np.append(Pd_OR,detect_OR/N)

  #plot
  plt.plot(
      SNR_db,
      Pd_OR,
      marker=markers[i],
      linestyle='dashed',
      label=str("K = "+str(k))
      )
  i += 1
# plot properties
plt.title("P_d v/s SNR for OR-based fusion")
plt.ylabel("Probability of Detection (P_d)")
plt.xlabel("Signal To Noise Ratio (dB)")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Simulation Pf
Pf_OR = np.array([])
Pf_AND = np.array([])
Pf_M = np.array([])

for snr_db in SNR_db:

  detect_OR = 0
  detect_AND = 0
  detect_M = 0

  print("At snr :",snr_db)

  Energy_stat_pre = np.array([])
  for _ in range(N):
    noise = np.random.randn(L,nodes)

    #signal = math.sqrt(10**(snr_db/10))*np.random.randn(L,nodes)

    Rec_signal = noise  # no signal
        
    Energy_stat = np.sum(np.abs(Rec_signal**2) ,axis=0) 
            
    if not Energy_stat_pre.any():
      Energy_stat_pre = Energy_stat

    detect = [detection(Energy_stat[x],Energy_stat_pre[x]) for x in range(nodes)]
    
    Energy_stat_pre = Energy_stat
    #print(detect)

    if detect_func_AND(detect):
      detect_AND += 1
        
    if detect_func_OR(detect):
      detect_OR += 1
    
    if detect_func_M(detect):
      detect_M += 1
      
  Pf_OR = np.append(Pf_OR,detect_OR/N)
  Pf_AND = np.append(Pf_AND,detect_AND/N)
  Pf_M = np.append(Pf_M,detect_M/N)

#plot
plt.plot(SNR_db,Pf_OR,'bo--',SNR_db,Pf_AND,'rx--',SNR_db,Pf_M,'kx--')
# plot properties
plt.xlabel("Signal To Noise Ratio (dB)")
plt.ylabel("Probability of False Alarm (P_f)")
plt.title("P_f v/s SNR")
plt.legend(["Pf_OR","Pf_AND","Pf_M"])
plt.grid()
plt.show()